In [1]:
import torch 
from datasets import concatenate_datasets, Dataset
import pandas as pd

In [2]:
tokens_to_be_added = []
with open('tokens_to_be_added.txt', 'r', encoding='utf-8') as f:
    for line in f:
        tokens_to_be_added.append(line.strip())

In [3]:
tokens_to_be_added

['<german_chancellor_angela>',
 '<德国总理安格拉•默克尔>',
 '<foreign_assets>',
 '<外国资产>',
 '<st_petersburg>',
 '<圣彼得堡>',
 '<interbank_lending_rates>',
 '<间拆借利率>',
 '<exiled_tibetan_spiritual_leader>',
 '<负责“利益”者>',
 '<economic_outlook>',
 '<经济前景>',
 '<coking_coal>',
 '<炼焦煤>',
 '<us_securities_exchange>',
 '<美国《SecuritiesExchange>',
 '<operating_profits>',
 '<营业利润>',
 '<chinese_telecoms_company_huawei>',
 '<中国电信设备商(华Huawei)>',
 '<share_price>',
 '<股价>',
 '<purchasing_managers_indices>',
 '<采购经理指数>',
 '<international_trade>',
 '<国际贸易>',
 '<dry_bulk>',
 '<干散货>',
 '<business_model>',
 '<商业模式>',
 '<market_interest_rates>',
 '<市场利率>',
 '<north_korean_supreme_leader>',
 '<朝鲜最高人>',
 '<silicon_valley>',
 '<硅谷>',
 '<business_education>',
 '<商业教育>',
 '<potential_buyers>',
 '<潜在买家>',
 '<federal_open_market>',
 '<联邦公开市场>',
 '<banking_crisis>',
 '<银行危机>',
 '<cut_us_interest_rates>',
 '<减息美联>',
 '<us_politics>',
 '<美国政治>',
 '<crown_prince_mohammed_bin>',
 '<王储穆罕默德•本bin>',
 '<uk_prime_minister_david>',
 '<英国首相

In [4]:
dataset = './training_data.csv'
dataset1 = pd.read_csv(dataset)
train_dataset = dataset1[:175000]
validation_dataset = dataset1[175000:200000]

train_dataset_hf = Dataset.from_pandas(train_dataset)
validation_dataset_hf = Dataset.from_pandas(validation_dataset)

In [5]:
from transformers import MBartForConditionalGeneration, MBartTokenizer
import torch

# Load mBART model and tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBartTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Add custom tokens and resize model embeddings
tokenizer.add_tokens(tokens_to_be_added)
model.resize_token_embeddings(len(tokenizer))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


MBartScaledWordEmbedding(255669, 1024, padding_idx=1)

In [11]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["zh"], max_length=1024, padding="max_length",truncation=True)

    target_encodings = tokenizer(example_batch["en"], max_length=1024, padding="max_length", truncation=True)

    return {"input_ids": input_encodings["input_ids"],
           "attention_mask": input_encodings["attention_mask"],
           "labels": target_encodings["input_ids"]}



In [13]:
train_dataset_tf = train_dataset_hf.map(convert_examples_to_features, batched=True, remove_columns=["zh","en"])
val_dataset_tf = validation_dataset_hf.map(convert_examples_to_features, batched=True, remove_columns=["zh","en"])

Map:   0%|          | 0/175000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [31]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer,TrainingArguments, Trainer

# Setup Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir='mbartTrans', 
    num_train_epochs=2,
    per_device_train_batch_size=1,  # Increased batch size
    per_device_eval_batch_size=1,
    evaluation_strategy='steps',
    save_strategy='no',
    eval_steps=2000,
    logging_steps=1000,
    weight_decay=0.01, 
    push_to_hub=False,
    learning_rate=2e-5,
    optim="adafactor",
    no_cuda=False  # Ensure CUDA is enabled when available
)

# Data collator for seq2seq tasks
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model, 
    args=training_args, 
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=train_dataset_tf,
    eval_dataset=val_dataset_tf
)

# Check CUDA availability and model device
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Model is on device: {next(model.parameters()).device}")

# Train the model
trainer.train()

# Save the model
trainer.save_model("zhenTranslationMbart")

C:\Users\Vincent\anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


CUDA available: True
Model is on device: cuda:0


Step,Training Loss,Validation Loss


KeyboardInterrupt: 